In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

## 1. Data Preprocessing

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float')/255.
x_test = x_test.astype('float')/255.
x_train = np.reshape(x_train, (60000, 784))
x_test = np.reshape(x_test, (10000, 784))
x_train.shape



## 2. Adding Noise 

In [ ]:
x_train_noisy = x_train + np.random.rand(60000, 784) * 0.9
x_test_noisy = x_test + np.random.rand(10000, 784) * 0.9
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
def plot(x,predictions, labels=False):
    plt.figure(figsize=(20,2))
    for i in range(10):
        plt.subplot(1,10,i+1)
        plt.imshow(x[i].reshape(28,28),cmap="binary")
        plt.xticks([])
        plt.yticks([])
        if labels:
            plt.xlabel(np.argmax(p[i]))
    plt.show()

We see the original images as follows:

In [ ]:
plot(x_train,None)

We see the noisy images as follows:

In [ ]:
plot(x_train_noisy,None)

## 3. Building the Autoencoder

In [ ]:
model = Sequential()
model.add(Dense(units=256,activation="relu", input_shape=(784,)))
model.add(Dense(units=256,activation="relu"))
model.add(Dense(units=10,activation="softmax"))
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(x=x_train, y=y_train,validation_data=(x_test,y_test), epochs=4)

In [ ]:
model.evaluate(x_test,y_test)

As seen our model's prediction is very low in the noisy images %97 versus %26

In [ ]:
model.evaluate(x_test_noisy,y_test)

## 4. Building the Autoencoder

<font color="green">
An autoencoder is an unsupervised learning technique for neural networks that learns efficient data representations (encoding) by training the network to ignore signal “noise.” Autoencoders can be used for image denoising, image compression, and, in some cases, even generation of image data.
  
Autoencoder gets the noisy images as input and the original images as output. This forces the model to learn the most important characteristics of the image like Principal Component Analysis.

In [ ]:
input_image = Input(shape=(784,))
encoded = Dense(units=64,activation="relu")(input_image) # This will reduce the dimensionality of the image and get the most important parts
decoded = Dense(units=784,activation="sigmoid")(encoded) # This will return 1 or 0 on the encoded pixels, so will reduce the noises.

autoencoder = Model(input_image, decoded)
autoencoder.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

## 5. Training the Autoencoder

In [ ]:
autoencoder.fit(x=x_train_noisy,y=x_train, epochs =100, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

print(' ***********************************************************************************')
print('Training is complete!')

## 6.Denoised Images and Evaluate Performance of the Model

In [ ]:
predictions = autoencoder.predict(x_test_noisy)

In [ ]:
plot(x_test_noisy, None)

In [ ]:
plot(predictions, None)

Lets see the performance of our classifier with the denoised images. As seen below, we have almost the same accuracy as we had with the original images.

In [ ]:
model.evaluate(predictions,y_test)

## 7. Composite Model

In [ ]:
noisy_image = Input(shape=(784,))
x = autoencoder(noisy_image)
y = model(x)

denoise_and_classify = Model(noisy_image, y)

In [ ]:
p = denoise_and_classify.predict(x_test_noisy)

In [ ]:
plot(x_test_noisy, p, True)

In [ ]:
plot(x_test_noisy, to_categorical(y_test), True)